# Imports and initialization

In [12]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Choose one of the two below cells depending on the environment.

In [13]:
# dir_path = './' #uncomment if in local environment

In [14]:
#uncomment in google colab environment
from google.colab import drive
drive.mount('/content/drive', force_remount=True) #path to root directory in Drive
dir_path = './drive/MyDrive/BAKA/' #colab
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', header=None, sep=' ')
if len(df.columns) > 31:
  df = df.drop(columns=[31])

Mounted at /content/drive


In [15]:
files = os.listdir(dir_path + 'DataCollection')
files

['4', '7', '8', '0', '5', '2', '3', '6', '1']

# Dataset Split
Split samples into Test sets and Training sets <br>
Use 20% of available samples for Testing and the remaining for Training

In [16]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = len(samples);
    shuffle(samples, random_state = 0)

    for k in range(0, num_tests):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[60:])
        if len(df.columns) > 31:
          df = df.drop(columns=[31])

        if df.isnull().values.any() or '-nan(ind)' in df.values:
          print(f'Found NaN value at gesture[{i}] index [{samples[k]}], sample is thrown away')
          continue
        
        x_train.append(df.to_numpy())
        y_train.append(int(i));
    
    print(len(samples), ' ', num_tests, ' ', len(samples)- num_tests)
x_train = np.array(x_train)
y_train = np.array(y_train);

print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


429   429   0
264   264   0
233   233   0
253   253   0
771   771   0
Found NaN value at gesture[2] index [618.txt], sample is thrown away
687   687   0
568   568   0
266   266   0
390   390   0
x_train.shape:  (3860, 60, 31)
y_train.shiape:  (3860,)


In [17]:
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)

x_train.shape:  (3860, 60, 31)
y_train.shape:  (3860,)


### Scaling and feature labels
Apply min-max scaling technique and relable features corresponding to the description in bachelor thesis's text

In [18]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

Form preprocessed datasets to corresponding shapes as well as shuffle samples

In [19]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)


In [20]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)

x_train.shape:  (3860, 60, 31)
y_train.shiape:  (3860,)


# Model definition

Define Two-layered bidirectional LSTM<br>
Each layer consists of `Bidirectional(LSTM)` cell with addition of `Dropout()` and `BatchNormalization()` to minimize overfitting and decrease learning time

In [21]:
model = Sequential()
model.add(Bidirectional(LSTM(units=60, return_sequences=True ,dtype='float64'),input_shape=x_train.shape[1:],dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.6))

model.add(Bidirectional(LSTM(units=60 ,dtype='float64') ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.6))

model.add(Dense(len(files), activation='softmax',dtype='float64'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 60, 120)           44160     
_________________________________________________________________
batch_normalization_2 (Batch (None, 60, 120)           480       
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 120)           0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 120)               86880     
_________________________________________________________________
batch_normalization_3 (Batch (None, 120)               480       
_________________________________________________________________
dropout_3 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 9)                

# Training

Model training using 200 epochs <br>
Marking checkpoints(models with best `val_accuracy`) to `./Checkpoints` directory

In [22]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-5)

checkpoint_filepath = dir_path + 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=200,
            batch_size=24,
            shuffle=True
         )

Epoch 1/200
161/161 [==============================] - 10s 31ms/step - loss: 2.6685 - accuracy: 0.2598
Epoch 2/200
161/161 [==============================] - 5s 31ms/step - loss: 1.4051 - accuracy: 0.5370
Epoch 3/200
161/161 [==============================] - 5s 31ms/step - loss: 0.9227 - accuracy: 0.6801
Epoch 4/200
161/161 [==============================] - 5s 31ms/step - loss: 0.6391 - accuracy: 0.7852
Epoch 5/200
161/161 [==============================] - 5s 31ms/step - loss: 0.5559 - accuracy: 0.8197
Epoch 6/200
161/161 [==============================] - 5s 31ms/step - loss: 0.4515 - accuracy: 0.8575
Epoch 7/200
161/161 [==============================] - 5s 31ms/step - loss: 0.3856 - accuracy: 0.8811
Epoch 8/200
161/161 [==============================] - 5s 31ms/step - loss: 0.3824 - accuracy: 0.8832
Epoch 9/200
161/161 [==============================] - 5s 31ms/step - loss: 0.3313 - accuracy: 0.8964
Epoch 10/200
161/161 [==============================] - 5s 31ms/step - loss: 0.31

In [23]:
model.save(dir_path + 'Models/gestures_bidir', save_format='tf')

INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_bidir/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_bidir/assets


In [24]:
min_max_scaler.data_min_

array([   0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,
          0.    ,    0.    ,    0.    ,    0.    , -693.223 ,  -12.7777,
       -612.731 , -680.146 ,    0.    , -584.665 , -668.353 ,    0.    ,
       -569.33  , -649.613 ,    0.    , -597.032 , -675.481 ,    0.    ,
       -590.976 , -662.551 ,    0.    , -574.437 ,    0.    ,    0.    ,
          0.    ])

In [25]:
min_max_scaler.data_max_

array([ 180.   ,  180.   ,  180.   ,  179.999,  179.997,  180.   ,
        179.988,  180.   ,  179.989,  179.96 ,  539.053, 1036.65 ,
        720.07 ,  581.654, 1090.93 ,  709.384,  585.01 , 1068.79 ,
        705.543,  567.779, 1056.79 ,  702.861,  544.45 , 1061.02 ,
        693.173,  499.893, 1008.3  ,  707.228,  145.94 ,  161.082,
        146.668])

In [26]:
model.input

<KerasTensor: shape=(None, 60, 31) dtype=float64 (created by layer 'bidirectional_2_input')>